## Searching the documents
Search the corresponding documents using the index document name. The search query is extracted from the original claim, by taking the capitalized words and nouns in the claim. Then the BM25 similarity is use to rank the documents, and top-3 documents is selected. The results are stored to a json file, to be used to get the candidate sentences.


In [10]:
import sys, os, lucene, threading, time, string
from datetime import datetime

from java.nio.file import Paths
from org.apache.lucene import analysis, document, index, queryparser, search, store, util
from org.apache.lucene.document import Document, Field, FieldType

from org.apache.lucene.store import SimpleFSDirectory
assert lucene.getVMEnv() or lucene.initVM()
from org.apache.lucene.search.similarities import BM25Similarity
import nltk
from nltk.tag import StanfordPOSTagger

import json

jar = '/Users/qijieli/Downloads/stanford-postagger-2018-10-16/stanford-postagger.jar'
model = '/Users/qijieli/Downloads/stanford-postagger-2018-10-16/models/english-bidirectional-distsim.tagger'

store = SimpleFSDirectory(Paths.get('/Users/qijieli/Desktop/web_project_code/doc_name_index'))
analyzer = analysis.standard.StandardAnalyzer()
ireader = index.DirectoryReader.open(store)
isearcher = search.IndexSearcher(ireader)
#use BM25
isearcher.setSimilarity(BM25Similarity())
# Parse a simple query that searches for "text":
parser = queryparser.classic.QueryParser('content', analyzer)

with open('test-unlabelled.json', 'r') as f:
    data = json.load(f)

print(len(data))
claim_doc_dict = {}
for claim_id in data.keys():
    claim_doc_dict[claim_id] = {}
    claim_doc_dict[claim_id]['claim'] = data[claim_id]['claim']

eng_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
start = time.perf_counter()
processed = 0

for claim_id in claim_doc_dict.keys():
    item = claim_doc_dict[claim_id]['claim']

    item = ''.join(c for c in item if c not in string.punctuation)
    final_q_list = []
    for word in item.split():
        if word[0].isupper():
            final_q_list.append(word)
    nouns = ["NN","NNS","NNP","NNPS","VBG"]
    q_list = eng_tagger.tag(item.split())
    
    for word in q_list:
        if word[1] in nouns:
            if word[0] not in final_q_list:
                final_q_list.append(word[0])
    q = ""
    for word in final_q_list:
        q = q + word + " "
            
    if len(q)<1:
        q = item
    print(q)
    
    
    
    query = parser.parse(q)
    hits = isearcher.search(query, 3).scoreDocs
#     print(hits)
        
        
# Iterate through the results:

    claim_doc_dict[claim_id]['candi_docs'] = []
    for hit in hits:

        hitDoc = isearcher.doc(hit.doc)
        claim_doc_dict[claim_id]['candi_docs'].append([hitDoc['docID'],int(hitDoc['wiki'])+1])
    
    print(claim_doc_dict[claim_id])
    
    processed += 1
    print(processed)
        
    
elapsed = (time.perf_counter() - start)
print("Time used:",elapsed)
print("==========================")
print(len(claim_doc_dict))

ireader.close()
store.close()
#write to a json file
json_str = json.dumps(claim_doc_dict)
with open('unlabl_docs_3.json', 'w') as json_file:
    json_file.write(json_str)

14997
RavenSymoné Anglican 
{'claim': 'Raven-Symoné is an Anglican.', 'candi_docs': [['Anglican_Papalism', 9], ['Anglican_chant', 9], ['Anglican_Journal', 9]]}
1
Temple Dog anniversary album 
{'claim': 'Temple of the Dog celebrated the 37th anniversary of their self-titled album.', 'candi_docs': [['Temple_of_the_Dog', 95], ['Temple_of_the_Dog_-LRB-album-RRB-', 94], ['10th_Anniversary_Album', 1]]}
2
Solanum plants flowers 
{'claim': 'Solanum contains plants with ornamental flowers.', 'candi_docs': [['Flowers_and_Plants_Association', 36], ['Solanum', 90], ['On_Plants', 73]]}
3
A Good Day Die Hard Library Congress part 
{'claim': 'A Good Day to Die Hard is part of the Library of Congress.', 'candi_docs': [['A_Good_Day_to_Die_Hard', 7], ['A_Good_Day_to_Die', 6], ['The_Hard_Part', 96]]}
4
The Icelandic Coast Guard Gaeslan infrastructure 
{'claim': 'The Icelandic Coast Guard is also known as Gaeslan and it is important to modern infrastructure.', 'candi_docs': [['Icelandic_Coast_Guard', 46],

The Love Club EP Lorde debut play 
{'claim': 'The Love Club EP is the debut extended play by Lorde.', 'candi_docs': [['The_Love_Club_EP', 97], ['Lorde', 60], ['The_Love_EP', 96]]}
42
See You Other Side book 
{'claim': 'See You on the Other Side is a book.', 'candi_docs': [['See_You_on_the_Other_Side', 89], ['See_You_on_the_Other_Side_World_Tour', 88], ['The_Other_Side', 98]]}
43
Java films 
{'claim': 'Java is in films.', 'candi_docs': [['Java', 48], ['Lief_Java', 58], ['Java_plum', 48]]}
44
William Howard Taft offers candy court 
{'claim': 'William Howard Taft declined offers of candy to a federal court.', 'candi_docs': [['William_Howard_Taft', 106], ['Taft_Court', 94], ['Inauguration_of_William_Howard_Taft', 47]]}
45
Sausage Party Barack Obama 
{'claim': 'Sausage Party was only written by Barack Obama.', 'candi_docs': [['Barack_Obama', 12], ['Family_of_Barack_Obama', 34], ['Inauguration_of_Barack_Obama', 48]]}
46
Cambridgeshires Cambridgeshire County Council Peterborough City Council 

Mandy Moore American 
{'claim': 'Mandy Moore is not an American.', 'candi_docs': [['Mandy_Moore', 61], ['Mandy_Moore_discography', 62], ['The_Best_of_Mandy_Moore', 95]]}
83
Veep Comedy award 
{'claim': 'Veep has not won an award in Comedy.', 'candi_docs': [['Veep', 103], ['Comedy_award', 23], ['Saint_Veep', 87]]}
84
James Brolin soap operas 
{'claim': 'James Brolin appeared in soap operas.', 'candi_docs': [['James_Brolin', 48], ['Armenian_soap_operas', 10], ['List_of_soap_operas', 61]]}
85
JeanJacques Dessalines September 
{'claim': 'Jean-Jacques Dessalines was conceived on September 20th, 1758.', 'candi_docs': [['Dessalines', 27], ['Dessalines_Arrondissement', 27], ['Jean-Jacques_Dessalines', 49]]}
86
Anneliese Pol van der career science 
{'claim': 'Anneliese van der Pol had an early career in science.', 'candi_docs': [['Anneliese_van_der_Pol', 11], ['Van_der_Pol', 102], ['Balthasar_van_der_Pol', 13]]}
87
JeanJacques Dessalines territory 
{'claim': 'Jean-Jacques Dessalines ruled over 

William Cohen politician 
{'claim': 'William Cohen is a politician.', 'candi_docs': [['William_Cohen', 106], ['David_William_Cohen', 27], ['William_Edgar_Cohen', 106]]}
124
William Cohen city 
{'claim': 'William Cohen is from a city.', 'candi_docs': [['William_Cohen', 106], ['David_William_Cohen', 27], ['William_Edgar_Cohen', 106]]}
125
Ronaldinho football matches 
{'claim': 'Ronaldinho has played 97 football matches.', 'candi_docs': [['Ronaldinho', 84], ['Ronaldinho_Gomes', 83], ['And_No_Matches', 9]]}
126
William Cohen Canadian 
{'claim': 'William Cohen is Canadian.', 'candi_docs': [['William_Cohen', 106], ['David_William_Cohen', 27], ['William_Edgar_Cohen', 106]]}
127
Volkswagen Group vehicles 
{'claim': 'Volkswagen Group designs only non-commercial vehicles.', 'candi_docs': [['Volkswagen_Commercial_Vehicles', 103], ['Volkswagen_Group', 103], ['List_of_Volkswagen_passenger_vehicles', 60]]}
128
Arjit Singh Indian media 
{'claim': 'Arjit Singh goes unmentioned in the Indian media.', '

Leslie Alexander stock broker 
{'claim': 'Leslie Alexander is a former stock broker.', 'candi_docs': [['Leslie_Alexander', 57], ['Cazenove_-LRB-stock_broker-RRB-', 21], ['Broker', 16]]}
167
Eric Bana dancer 
{'claim': 'Eric Bana is a dancer.', 'candi_docs': [['Eric_Bana', 32], ['Eric_Dancer', 32], ['Bana_Bana', 12]]}
168
The Far Madding Crowd Spanish cinematography film director 
{'claim': 'The cinematography of Far from the Madding Crowd was by an Spanish film director.', 'candi_docs': [['Far_from_the_Madding_Crowd', 33], ['The_Madding_Crowd', 96], ['Far_from_the_Madding_Crowd_-LRB-1915_film-RRB-', 34]]}
169
The Beverly Hillbillies clothing store 
{'claim': 'The Beverly Hillbillies is a clothing store.', 'candi_docs': [['The_Beverly_Hillbillies', 95], ['Return_of_the_Beverly_Hillbillies', 81], ['List_of_The_Beverly_Hillbillies_episodes', 60]]}
170
Sausage Party men 
{'claim': 'Sausage Party was directed by two men.', 'candi_docs': [['Sausage_Party', 86], ['Sausage', 85], ['Small_sausa

...

To Pimp Butterfly America copies 
{'claim': 'To Pimp a Butterfly refused to sell any copies in America until 2017.', 'candi_docs': [['To_Pimp_a_Butterfly', 98], ['Leica_copies', 57], ['The_Pimp', 96]]}
14787
Bullitt Peter Yates action film 
{'claim': 'Bullitt is an action film directed by Peter Yates.', 'candi_docs': [['Peter_Yates', 78], ['Bullitt', 19], ['Peter_W._Yates', 78]]}
14788
Colombiana genre 
{'claim': 'Colombiana is of a genre.', 'candi_docs': [['A._colombiana', 6], ['Colombiana', 24], ['Genre', 38]]}
14789
DJ Quik American 
{'claim': 'DJ Quik is an American.', 'candi_docs': [['DJ_Quik', 25], ['DJ_Quik_discography', 26], ['DJ_Quik_production_discography', 25]]}
14790
The Godfather Part II TV show 
{'claim': 'The Godfather Part II is a TV show.', 'candi_docs': [['The_Godfather_Part_II', 97], ['The_Godfather_Part_III', 97], ['The_Godfather_Part_II_-LRB-soundtrack-RRB-', 96]]}
14791
Jeong Hyeongdon ABC Entertainment 
{'claim': 'Jeong Hyeong-don has worked under ABC Entertainme

Luke Cage part cult 
{'claim': 'Luke Cage was part of a cult.', 'candi_docs': [['Luke_Cage', 62], ['List_of_Luke_Cage_characters', 57], ['Luke_Cage_-LRB-soundtrack-RRB-', 62]]}
14833
Jeong Hyeongdon Beijing 
{'claim': 'Jeong Hyeong-don was born in Beijing.', 'candi_docs': [['Jeong', 49], ['Jeong_Ho-jeong', 50], ['Beijing', 14]]}
14834
Paramore rock band 
{'claim': 'Paramore is a classic rock band.', 'candi_docs': [['Paramore', 75], ['Rock_Band', 85], ['Jim_Paramore', 50]]}
14835
Harris Jayaraj CEO 
{'claim': 'Harris Jayaraj is a CEO.', 'candi_docs': [['Harris_Jayaraj', 42], ['Jayaraj', 50], ['The_CEO', 94]]}
14836
The Faroe Islands part tour 
{'claim': 'The Faroe Islands were part of the tour.', 'candi_docs': [['Tour_of_Faroe_Islands', 98], ['Faroe_Islands', 34], ['List_of_islands_of_the_Faroe_Islands', 61]]}
14837
Gin wine 
{'claim': 'Gin is a wine.', 'candi_docs': [['Gin_Gin', 38], ['Gin', 39], ['Gin_Gin,_Queensland', 38]]}
14838
Fidel Castro President Cuba elections 
{'claim': 'Fide

Phillip Glass eleven concertos 
{'claim': 'Phillip Glass has written eleven popular concertos.', 'candi_docs': [['Eleven_Eleven', 33], ['Brandenburg_Concertos', 17], ['Strathclyde_Concertos', 92]]}
14876
Mud American actors 
{'claim': 'Mud stars multiple American actors.', 'candi_docs': [['American_Federation_of_Actors', 9], ['Lists_of_American_actors', 59], ['The_Actors', 94]]}
14877
Mary Teck queenconsort 
{'claim': 'Mary of Teck was queen-consort.', 'candi_docs': [['Mary_of_Teck', 64], ['List_of_titles_and_honours_of_Mary_of_Teck', 61], ['Wedding_dress_of_Princess_Mary_of_Teck', 106]]}
14878
Connie Nielsen Boss television actress 
{'claim': 'Connie Nielsen is a television actress known for Boss.', 'candi_docs': [['Connie_Nielsen', 22], ['Connie', 24], ['Be_the_Boss', 14]]}
14879
Taylor Lautner milk carton 
{'claim': 'Taylor Lautner appeared on a milk carton.', 'candi_docs': [['Taylor_Lautner', 94], ['Photo_on_a_milk_carton', 77], ['The_Face_on_the_Milk_Carton', 97]]}
14880
Anne Sull

Hourglass Australian singersongwriter 
{'claim': 'Hourglass is performed by an Australian singer-songwriter.', 'candi_docs': [['Hourglass', 44], ['Hourglass_Buttress', 44], ['Hourglass_Federalism', 45]]}
14920
Henry II France cause war religion 
{'claim': 'Henry II of France was not involved with the cause of a war about religion.', 'candi_docs': [['Henry_II_of_France', 44], ['Religion_in_France', 82], ['The_Religion_War', 97]]}
14921
Wentworth Miller Stoker screenwriting debut 
{'claim': 'Wentworth Miller made his screenwriting debut with the 2013 Stoker.', 'candi_docs': [['Wentworth_Miller', 107], ['Screenwriting', 87], ['Stoker', 93]]}
14922
The NAACP Image Award Outstanding Supporting Actor Drama Series Joe Morton 
{'claim': 'The NAACP Image Award for Outstanding Supporting Actor in a Drama Series has been won by Joe Morton.', 'candi_docs': [['NAACP_Image_Award_for_Outstanding_Supporting_Actor_in_a_Drama_Series', 68], ['NAACP_Image_Award_for_Outstanding_Actor_in_a_Drama_Series', 68

A River Runs Through It crisis 
{'claim': 'A River Runs Through It is a crisis.', 'candi_docs': [['A_River_Runs_Through_It', 7], ['A_River_Runs_Through_Bob', 6], ['A_River_Runs_Through_It_-LRB-film-RRB-', 6]]}
14963
Regina King The Leftovers crew member 
{'claim': 'Regina King is a crew member of The Leftovers.', 'candi_docs': [['Regina_King', 81], ['Leftovers', 58], ['The_Leftovers', 96]]}
14964
Kellyanne Conway phrase facts 
{'claim': 'Kellyanne Conway has used the phrase "alternative facts."', 'candi_docs': [['Kellyanne_Conway', 53], ['Kellyanne_Farquhar', 53], ['Phrase', 77]]}
14965
Star Trek Discovery Star Trek media franchise 
{'claim': 'Star Trek: Discovery is directly connected to the Star Trek media franchise.', 'candi_docs': [['Star_Trek', 92], ['Star_Trek-COLON-_Discovery', 93], ['Law_in_Star_Trek', 56]]}
14966
Alice Cooper United States 
{'claim': 'Alice Cooper was raised in the United States.', 'candi_docs': [['Alice_Cooper', 8], ['Alice_Cooper_and_Friends', 7], ['Alice_Co